## Copy the output from peersim to convert to .txt file

In [ ]:
# just copy the o/p here
data = """
Current time: 200000
Hit Rate Avg: 0.2782738095238094 Network Load Avg: 0.00741199999999995 Messages: 1853.0
Current time: 300000
Hit Rate Avg: 0.36964697237335875 Network Load Avg: 0.01722399999999993 Messages: 4324.0
Current time: 400000
Hit Rate Avg: 0.4011483368404285 Network Load Avg: 0.034947999999999924 Messages: 8839.0
Current time: 500000
Hit Rate Avg: 0.41064700972549867 Network Load Avg: 0.060680000000000005 Messages: 15558.0
Current time: 600000
Hit Rate Avg: 0.4089157883191851 Network Load Avg: 0.093608 Messages: 24488.0
Current time: 700000
Hit Rate Avg: 0.411039969436102 Network Load Avg: 0.132196 Messages: 35375.0
Current time: 800000
Hit Rate Avg: 0.4107319141547009 Network Load Avg: 0.17707599999999996 Messages: 48677.0
Current time: 900000
Hit Rate Avg: 0.41033795489429886 Network Load Avg: 0.22601199999999994 Messages: 64048.0
Current time: 1000000
Hit Rate Avg: 0.4104354364195713 Network Load Avg: 0.278332 Messages: 81582.0
Current time: 1100000
Hit Rate Avg: 0.41135355210320185 Network Load Avg: 0.32891600000000015 Messages: 100002.0
Current time: 1200000
Hit Rate Avg: 0.41140424416478266 Network Load Avg: 0.3754760000000001 Messages: 118248.0
Current time: 1300000
Hit Rate Avg: 0.4113160907782131 Network Load Avg: 0.4140280000000002 Messages: 134458.0
Current time: 1400000
Hit Rate Avg: 0.41191885882745716 Network Load Avg: 0.4451839999999999 Messages: 148353.0
Current time: 1500000
Hit Rate Avg: 0.4117662629194911 Network Load Avg: 0.47070399999999973 Messages: 160387.0
Current time: 1600000
Hit Rate Avg: 0.41182815420485314 Network Load Avg: 0.49076399999999937 Messages: 170219.0
Current time: 1700000
Hit Rate Avg: 0.411925243311749 Network Load Avg: 0.5053440000000002 Messages: 177760.0
Current time: 1800000
Hit Rate Avg: 0.4122635787987705 Network Load Avg: 0.515452 Messages: 183135.0
Current time: 1900000
Hit Rate Avg: 0.41216518205939917 Network Load Avg: 0.5215079999999999 Messages: 186335.0
Current time: 2000000
Hit Rate Avg: 0.412222296854373 Network Load Avg: 0.5234479999999998 Messages: 187432.0
Current time: 2100000
Hit Rate Avg: 0.412222296854373 Network Load Avg: 0.5234479999999998 Messages: 187432.0
Current time: 2200000
Hit Rate Avg: 0.412222296854373 Network Load Avg: 0.5234479999999998 Messages: 187432.0
Current time: 2300000
Hit Rate Avg: 0.412222296854373 Network Load Avg: 0.5234479999999998 Messages: 187432.0
Current time: 2400000
Hit Rate Avg: 0.412222296854373 Network Load Avg: 0.5234479999999998 Messages: 187432.0
"""

# path to ASCII file you want to create from o/p
output_file_path = 'raw.txt'

with open(output_file_path, 'w') as file:
    file.write(data)

print(f"Data written to {output_file_path}")


Data written to raw.txt


## Convert historical data to csv file

In [ ]:
import re
import pandas as pd

def parse_ascii_to_csv(ascii_file_path, csv_file_path):
    with open(ascii_file_path, 'r') as file:

        extracted_data = []
        for line in file:

            if line.startswith('Hit Rate'):
                data_values = re.findall(r"[-+]?\d*\.\d+|\d+", line)
                extracted_data.append(data_values)

        df = pd.DataFrame(extracted_data, columns=['Hit Rate', 'Network Load', 'Num Queries'])

        df.to_csv(csv_file_path, index=False)

parse_ascii_to_csv('raw.txt', 'parse.csv')


## Preprocess data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

def preprocess_data(input_csv_path):
    data = pd.read_csv(input_csv_path)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data[['Hit Rate', 'Network Load', 'Num Queries']])
    scaled_df = pd.DataFrame(scaled_data, columns=['Hit Rate', 'Network Load', 'Num Queries'])
    return scaled_df, scaler

def create_sequences(df, n_input):
    X, y = [], []
    for i in range(len(df) - n_input):
        X.append(df.iloc[i:i+n_input].values)
        y.append(df.iloc[i+n_input].values)
    return np.array(X), np.array(y)

def adjust_parameters_v3(metrics, current_params):
    q1, q2, low, high = current_params
    hit_rate, net_load, num_queries = metrics

    # Adjust q1 and q2 based on hit rate and number of queries
    if hit_rate > 0.7:  # High success rate
        q1 += 10  # Strengthen pheromone increase
        q2 += 0.2  # Sharpen pheromone sensitivity
    elif hit_rate < 0.2:  # Low success rate
        q1 -= 10
        q2 -= 0.05

    # Modify low and high thresholds based on network load
    if net_load > 0.05:
        low += 0.05
        high -= 0.05
    else:
        low -= 0.05
        high += 0.05

    # Normalize values to ensure they remain within acceptable ranges
    q1 = max(0, min(100, q1))
    q2 = max(0, max(1, q2))
    low = max(0, min(1, low))
    high = max(0, min(1, high))

    return q1, q2, low, high


input_csv_path = 'parse.csv'
df, scaler = preprocess_data(input_csv_path)

n_input = 10  # number of time steps
n_features = 3  #Hit Rate, Network Load, Num Queries

X, y = create_sequences(df, n_input)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Run LSTM

In [ ]:
model = Sequential([
    LSTM(100, activation='relu', input_shape=(n_input, n_features)),
    Dense(n_features)
])
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), verbose=1)

# Predict the latest data
latest_data = df.tail(n_input).values.reshape((1, n_input, n_features))
predicted_metrics = model.predict(latest_data, verbose=0)
print("Predicted Metrics:", predicted_metrics)

# Adjust parameters based on predictions
adjusted_params = adjust_parameters_v3(predicted_metrics[0], (80, 1, 0.05, 0.2))  # Also add initial parameters
print("Adjusted Parameters (q1, q2, low, high):", adjusted_params)

Epoch 1/50
1/1 [==============================] - 3s 3s/step - loss: 0.8405 - val_loss: 0.8609
Epoch 2/50
1/1 [==============================] - 0s 48ms/step - loss: 0.7982 - val_loss: 0.8154
Epoch 3/50
1/1 [==============================] - 0s 49ms/step - loss: 0.7564 - val_loss: 0.7702
Epoch 4/50
1/1 [==============================] - 0s 51ms/step - loss: 0.7147 - val_loss: 0.7248
Epoch 5/50
1/1 [==============================] - 0s 53ms/step - loss: 0.6730 - val_loss: 0.6784
Epoch 6/50
1/1 [==============================] - 0s 65ms/step - loss: 0.6306 - val_loss: 0.6318
Epoch 7/50
1/1 [==============================] - 0s 71ms/step - loss: 0.5880 - val_loss: 0.5848
Epoch 8/50
1/1 [==============================] - 0s 53ms/step - loss: 0.5449 - val_loss: 0.5367
Epoch 9/50
1/1 [==============================] - 0s 58ms/step - loss: 0.5009 - val_loss: 0.4877
Epoch 10/50
1/1 [==============================] - 0s 64ms/step - loss: 0.4562 - val_loss: 0.4374
Epoch 11/50
1/1 [==============

Predicted Metrics: [[1.049451  1.0290703 1.108806 ]]
Adjusted Parameters (q1, q2, low, high): (90, 1.05, 0.1, 0.15000000000000002)
